In [ ]:
#try to figuree out about indexing for group by
#group by merchant category as well so for example we have travel category and we should be able to show the divisons of diff airlines and stuff 
#keep track of mcg table as well from dontu sir
#voice commands support for the visualization part
#spherical data visulation

### **Import Libraries:**

In [41]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder, FunctionTransformer, LabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline # Note that this Pipeline is imported from imblearn, not sklearn
from sklearn.model_selection import train_test_split

### **Set Pandas Display Options:**

In [ ]:
pd.set_option('display.max_columns', None)

### Connect to SQLite Database:

In [ ]:
conn = sqlite3.connect(r"C:\Users\yashu\OneDrive\Desktop\Github_Projects\Credit_Card_Fraud_Detection\corecard_credit_card_transactions.db")

### Load Data from Database:

In [ ]:
df = pd.read_sql("SELECT * FROM transactions", conn)

### Database Visualization and Info:

In [ ]:
df

In [ ]:
for column in df.columns:
    print(f"Unique values in {column}: {df[column].unique()}")
    print(f"Number of unique values in {column}: {df[column].nunique()}")

In [ ]:
df.isnull().sum()

In [ ]:
# Identify columns with all null values
all_null_columns = df.columns[df.isnull().all()]
print(f"Columns with all values null: {all_null_columns}")
print(f"Number of columns with all values null: {len(all_null_columns)}")

In [ ]:
df.columns.unique()

In [ ]:
df.shape

### Fraud Only Database and Info:

In [ ]:
# Handle missing values in the 'FraudIndicator' column
df['FraudIndicator'] = df['FraudIndicator'].fillna(0)

fraud = pd.read_sql("SELECT * FROM transactions WHERE FraudIndicator = 1", conn)                                  
fraud

In [ ]:
#number of fraud cases
535908-535470

In [ ]:
#cross checking
count_of_fraud_equal_to_1 = sum(df['FraudIndicator'] == 1.0)
print(f"Count of fraud equal to 1: {count_of_fraud_equal_to_1}")

In [ ]:
df['FraudIndicator'].unique()

### Training and Validation Database (90% of Original Database) :

In [ ]:
data = pd.read_sql("SELECT * FROM transactions ORDER BY TranTime LIMIT ROUND((SELECT COUNT(*) FROM transactions) * 0.9)", conn)
data

In [ ]:
data.shape

In [ ]:
# Handle missing values in the 'FraudIndicator' column
data['FraudIndicator'] = data['FraudIndicator'].fillna(0)

In [ ]:
#cross checking
count_of_fraud_equal_to_1 = sum(data['FraudIndicator'] == 1.0)
print(f"Count of fraud equal to 1: {count_of_fraud_equal_to_1}")

### Preprocessing Data:

In [ ]:
# Split data into feature matrix X and target variable y
X = data.drop('FraudIndicator', axis=1)
y = data['FraudIndicator']

# Define numerical, categorical, and nominal data columns
numerical_data = ['TransactionAmount', 'OutstandingAmount', 'CurrentBalance', 'TotalOutStgAuthAmt', 'ActualReversalAmount', 'CalcOTB', 'ATC']
categorical_data = ['TranType', 'PrimaryCurrencyCode', 'MessageTypeIdentifier', 'ProcCode', 'ProcCodeFromAccType', 'MerchantType', 'IResponseCode', 'ResponseCode', 'AuthStatus', 'POSTransactionStatusInd', 'TxnCategory', 'AdvReasonCode', 'AVResponse', 'PostingRef', 'TerminalType', 'Field112', 'AuthVarianceException', 'TxnSubCategory', 'PinExist', 'CardAcceptorNameLocation', 'CrossBorderTxnIndicator', 'MerchantName']
nominal_data = ['TxnAcctId', 'UniqueID', 'AccountNumber']
date_time = ['TranTime', 'PostTime', 'TransmissionDateTime', 'TimeLocalTransaction', 'DateLocalTransaction', 'EffectiveDate_ForAgeOff', 'PurgeDate']

print(f"Number of numerical data columns: {len(numerical_data)}")
print(f"Number of categorical data columns: {len(categorical_data)}")
print(f"Number of nominal data columns: {len(nominal_data)}")
print(f"Number of date_time columns: {len(date_time)}")

# Define preprocessing pipelines
num_pipe = Pipeline([
    ('num_imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())  
])

cat_pipe = Pipeline([
    ('cat_imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessing = ColumnTransformer([
    ('num', num_pipe, numerical_data),
    ('cat', cat_pipe, categorical_data)
])


### Train and Evaluate the Model:

In [42]:
# Initialize the classifier
from sklearn.ensemble import RandomForestClassifier

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10, shuffle=True)

# Create the pipeline
model = ImbPipeline([
    ('preprocess', preprocessing),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier())
])

# Train the model
model.fit(X_train, y_train)

# Evaluate the model on the test set
accuracy = model.score(X_test, y_test)
print(f"Model accuracy: {accuracy}")

# Make predictions
y_predicted = model.predict(X_test)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'SMOTE(random_state=42)' (type <class 'imblearn.over_sampling._smote.base.SMOTE'>) doesn't

### Calculate Metrics:

In [ ]:
# Calculate the F1 score
f1_scores = f1_score(y_test, y_predicted, average=None)
binary_f1_score = f1_score(y_test, y_predicted, average='binary')
print(f"F1 scores for each class: {f1_scores}")
print(f"Binary F1 score: {binary_f1_score}")


In [ ]:
# Calculate probs
from sklearn.metrics import average_precision_score, precision_recall_curve


probs = model.predict_proba(X_test)

# Calculate average precision 
average_precision = average_precision_score(y_test, y_predicted)

# Obtain precision and recall 
precision, recall, _ = precision_recall_curve(y_test, y_predicted)

# Plot the recall precision tradeoff
#plot_pr_curve(recall, precision, average_precision)

# Print the roc auc score, the classification report and confusion matrix
print("auc roc score: ", roc_auc_score(y_test, probs[:,1]))
print('Classifcation report:\n', classification_report(y_test, y_predicted))

In [ ]:
# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_predicted)

# Extract true positives, true negatives, false positives, and false negatives from the confusion matrix
tn, fp, fn, tp = cm.ravel()

# Calculate metrics in terms of percentages
accuracy = accuracy_score(y_test, y_predicted) * 100
precision = precision_score(y_test, y_predicted) * 100
recall = recall_score(y_test, y_predicted) * 100
f1 = f1_score(y_test, y_predicted) * 100

# Print the metrics
print(f"Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.2f}%")
print(f"Recall: {recall:.2f}%")
print(f"F1-Score: {f1:.2f}%")

# Display the confusion matrix
print("Confusion Matrix:")
print(cm)

In [ ]:
# Display the confusion matrix
class_names = [False, True]
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(cm), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion Matrix')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
plt.show()

### Additional Queries and Data Analysis:

In [ ]:
# Query for merchant types and total transaction amounts
merchant_types_query = "SELECT MerchantType, COUNT(1), SUM(TransactionAmount) as TotalAmount FROM transactions GROUP BY MerchantType ORDER BY TotalAmount DESC"
merchant_types_data = pd.read_sql(merchant_types_query, conn)
print(merchant_types_data)

# Query for transactions with missing MerchantType
missing_merchant_type_query = "SELECT MessageTypeIdentifier, * FROM transactions WHERE MerchantType IS NULL"
missing_merchant_type_data = pd.read_sql(missing_merchant_type_query, conn)
print(missing_merchant_type_data)

In [37]:
data['FraudIndicator'] = data['FraudIndicator'].fillna(0)
# Split data into feature matrix X and target variable y
X = data.drop('FraudIndicator', axis=1)
y = data['FraudIndicator']

# Define numerical, categorical, and nominal data columns
numerical_data = ['TransactionAmount', 'OutstandingAmount', 'CurrentBalance', 'TotalOutStgAuthAmt', 'ActualReversalAmount', 'CalcOTB', 'ATC']
categorical_data = ['TranType', 'PrimaryCurrencyCode', 'MessageTypeIdentifier', 'ProcCode', 'ProcCodeFromAccType', 'MerchantType', 'IResponseCode', 'ResponseCode', 'AuthStatus', 'POSTransactionStatusInd', 'TxnCategory', 'AdvReasonCode', 'AVResponse', 'PostingRef', 'TerminalType', 'Field112', 'AuthVarianceException', 'TxnSubCategory', 'PinExist', 'CardAcceptorNameLocation', 'CrossBorderTxnIndicator', 'MerchantName']
nominal_data = ['TxnAcctId', 'UniqueID', 'AccountNumber']
date_time = ['TranTime', 'PostTime', 'TransmissionDateTime', 'TimeLocalTransaction', 'DateLocalTransaction', 'EffectiveDate_ForAgeOff', 'PurgeDate']

# Define preprocessing pipelines
num_pipe = Pipeline([
    ('num_imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())  
])

cat_pipe = Pipeline([
    ('cat_imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessing = ColumnTransformer([
    ('num', num_pipe, numerical_data),
    ('cat', cat_pipe, categorical_data)
])

x = preprocessing.transform(data)